# Q-Learner

**Overview**

Give an over view of q-learning. State, Action, State, Next State. Make this quite wordy since the rest of the content is based on knowing this fact

**Discrete Environment**

In [ ]:
# Create 3 to 4 q-learning problems

**Continuous Environment**

Explain this section. Use discretization and then show its limitations

In [ ]:
# Create a problem that shows discretization. Possible have it generate numbers and show the bins